# Ondes de Rossby des moyennes latitudes/diagrammes de Hovmöller 


**Auteur du calepin : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Tracé de cartes, animations et diagrammes de Hovmöller à partir de données réanalyses quotidiennes du géopotentiel à 500 hPa (données NCEP : ftp://ftp2.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/)

Concepts Python illustrés :

- Exploitation de fichiers de données quotidiennes au format netcdf (module xarray)
- Calcul d'anomalies quotidiennes (via la méthode groupby de xarray)
- Tracé de cartes et d'animations (matplotlib/cartopy)
- Réalisation de diagramme de Hovmöller

In [ ]:
%matplotlib inline

import os

import xarray as xr
import numpy as np

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

In [ ]:
dir_data='./data/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

# Ouverture des données

In [ ]:
year1='1980'
year2='2020'

fz    = xr.open_dataset('./data/hgt500.1980-2021.nc').sel(lat=slice(90,0)).sel(time=slice(year1,year2))
print(fz)

print(' ----- Computing daily anomalies ----- ')
fz_anom=fz.groupby('time.dayofyear') - fz.groupby('time.dayofyear').mean('time')
print(' ----- Done ----- ')

# Etude d'une séquence de cartes

In [ ]:
date1='2020-01-01'
date2='2020-12-31'

fz_days=fz.sel(time=slice(date1,date2))
fz_days_anom=fz_anom.sel(time=slice(date1,date2))

time  = fz_days.time.values

dataz1 = fz_days['hgt']
dataz2 = fz_days_anom['hgt']

In [ ]:
time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]

for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]

print(date_str)

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
fz_days_anom_avg=fz_days_anom.sel(lat=slice(60,40)).mean('lat')
profilesz = fz_days_anom_avg['hgt']
profilesz = lonflip(profilesz)

In [ ]:
dataz1=lonflip(dataz1)
dataz2=lonflip(dataz2)
lat  = dataz1.lat.values
lon  = dataz1.lon.values

In [ ]:
levels_z1 = np.arange(4800,6050,50)
levels_z2 = np.arange(-500,550,50)
cmap_z1='jet'
cmap_z2='RdBu_r'

In [ ]:
def plot_background(ax):
    ax.set_xticks(np.linspace(-180, 180, 13), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    ax.axes.axis('tight')
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.coastlines()
    return ax

In [ ]:
proj=ccrs.PlateCarree()

latS=30
latN=70
bounds = [(-180, 180, latS, latN)]

plt_title1 = 'Geopotential height (mgp) at 500 hPa : '

for i in range(len(time)):
    print(date_str[i])
    fig = plt.figure(figsize=(15., 12.))
    fig.suptitle(plt_title1+date_str[i], fontsize=16)
    ax = fig.add_subplot(3, 1, 1, projection=proj)
    ax.set_title('Daily field', loc='center')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, dataz1[i,:,:], levels_z1, transform=ccrs.PlateCarree(), cmap=cmap_z1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
    cb.set_label('mgp', size='large')
    
    ax = fig.add_subplot(3, 1, 2, projection=proj)
    ax.set_title('Anomaly', loc='center')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, dataz2[i,:,:], levels_z2, transform=ccrs.PlateCarree(), cmap=cmap_z2, extend='both')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
    cb.set_label('mgp', size='large')
    
    ax = fig.add_subplot(3,1,3)
    plt.title('Anomaly field : 40N-60N average', fontsize=12)
    ax.set_xticks(np.linspace(-180, 180, 13))
    ax.set_xticklabels(np.linspace(-180, 180, 13))
    plt.xlim(-180, lon[-1])
    plt.ylim(-320, 320)   
    plt.xlabel('Longitude')
    plt.ylabel('Anomaly (mgp')
    prof = ax.plot(lon, profilesz[i,:], color='black', linewidth=1, linestyle='-')
    plt.axhline(0, color='k')
    plt.axvline(0, color='k')
    plt.axvline(-90, color='k')
    plt.axvline(90, color='k')
    plt.close()
    
    figname=dir_anim+'Z500_'+date_str[i]
    fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format(dir_anim)))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
gif_filepath = dir_anim+'Z500_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
print("Original shape -", dataz1.shape)
lon0 = dataz1.coords['lon']
lon_idx = dataz1.dims.index('lon')
dataz1, lon2 = add_cyclic_point(dataz1.values, coord=lon0, axis=lon_idx)
dataz2, lon2 = add_cyclic_point(dataz2.values, coord=lon0, axis=lon_idx)
print("New shape -", dataz1.shape)

In [ ]:
proj=ccrs.NorthPolarStereo()
bounds = [(-180, 180, 0, 90)]

plt_title1 = 'Geopotential height (mgp) at 500 hPa : '

for i in range(len(time)):
    print(date_str[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle(plt_title1+date_str[i], fontsize=16)
    ax = fig.add_subplot(1, 2, 1, projection=proj)
    ax.set_title('Daily field', loc='center')
    ax.coastlines()
    
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    cf = ax.contourf(lon2, lat, dataz1[i,:,:], levels_z1, transform=ccrs.PlateCarree(), cmap=cmap_z1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    ax = fig.add_subplot(1, 2, 2, projection=proj)
    ax.set_title('Anomaly', loc='center')
    ax.coastlines()
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon2, lat, dataz2[i,:,:], levels_z2, transform=ccrs.PlateCarree(), cmap=cmap_z2, extend='both')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    plt.close()
    
    figname=dir_anim+'Z500_polar_'+date_str[i]
    fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
gif_filepath = dir_anim+'Z500_polar_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
levels_z_hov = np.arange(-280,320,40)
levels_v_hov = np.arange(-50,55,5)

In [ ]:
plt_title = 'Geopotential height at 500 hPa (anomaly) : '+date1+'-'+date2

fig = plt.figure(figsize=(12., 15.))
ax=plt.subplot(1, 1, 1)
plt.gca().invert_yaxis()
ax.set_yticks(time[::30])
ax.set_yticklabels(date_str[::30])
ax.set_xticklabels('')
cf = ax.contourf(lon, time, profilesz, levels_z_hov, cmap=cmap_z2, extend='both')

cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('mgp', size='large')
ax.set_title(plt_title, loc='center')

ax_inset = fig.add_axes([0.125, 0.03, 0.62, 0.05], projection=ccrs.PlateCarree(central_longitude=0.0))
bounds = [(lon[0], lon[-1], 40, 60)]
ax_inset.axes.axis('tight')
plot_background(ax_inset)
ax_inset.coastlines()
ax_inset.stock_img()
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())

plt.show()

figname=dir_figs+'z500_Hov_'+date1+'-'+date2
fig.savefig(figname+'.png', bbox_inches='tight')